## Visualizing crime statistics in LA

In [1]:
import pandas as pd

In [3]:
path = 'los-angeles-crime-arrest-data/'
cri = pd.read_csv(path + 'crime-data-from-2010-to-present.csv', encoding='utf-8')
arr = pd.read_csv(path + 'arrest-data-from-2010-to-present.csv', encoding='utf-8')

In [4]:
cri.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,102005556,2010-01-25T00:00:00,2010-01-22T00:00:00,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"{'latitude': '34.0454', 'needs_recoding': Fals..."
1,101822289,2010-11-11T00:00:00,2010-11-10T00:00:00,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"{'latitude': '33.9572', 'needs_recoding': Fals..."
2,101105609,2010-01-28T00:00:00,2010-01-27T00:00:00,2230,11,Northeast,1125,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,YORK,AVENUE 51,"{'latitude': '34.1211', 'needs_recoding': Fals..."
3,101620051,2010-11-11T00:00:00,2010-11-07T00:00:00,1600,16,Foothill,1641,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,EL DORADO,TRUESDALE,"{'latitude': '34.241', 'needs_recoding': False..."
4,101910498,2010-04-07T00:00:00,2010-04-07T00:00:00,1600,19,Mission,1902,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,GLENOAKS,DRELL,"{'latitude': '34.3147', 'needs_recoding': Fals..."


In [12]:
cri.shape[0]/1000000

1.993259

In [8]:
arr.shape[0]/1000000

1.27616

In [14]:
cri['Crime Code Description'].value_counts() / cri['Crime Code Description'].value_counts().sum()

BATTERY - SIMPLE ASSAULT                                    9.052210e-02
BURGLARY FROM VEHICLE                                       7.698498e-02
VEHICLE - STOLEN                                            7.606739e-02
THEFT PLAIN - PETTY ($950 & UNDER)                          7.098375e-02
BURGLARY                                                    7.070130e-02
THEFT OF IDENTITY                                           6.062183e-02
INTIMATE PARTNER - SIMPLE ASSAULT                           5.413245e-02
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)     5.146797e-02
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT              4.356132e-02
VANDALISM - MISDEAMEANOR ($399 OR UNDER)                    4.336617e-02
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)             4.153550e-02
ROBBERY                                                     3.983025e-02
THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD    3.515900e-02
CRIMINAL THREATS - NO WEAPON DISPLAYED             